In [34]:
import os
import json

import numpy as np
import pandas as pd

import re
from utils import process_basic, process_detail, election, monetary

from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("hyunwoongko/kobart")

In [30]:
data_path='/content/drive/MyDrive/Dacon/한국어 생성요약/data/235813_AI 기반 회의 녹취록 요약 경진대회_data/'
model_path='/content/drive/MyDrive/Dacon/한국어 생성요약/LG_model/'
sub_path='/content/drive/MyDrive/Dacon/한국어 생성요약/LG_sub/'

# Make train data

In [31]:
train=pd.read_json(data_path+'train.json')
train.head()

,id,title,region,num_agenda,context,label
0,1000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,4,{'AGENDA_1': {'0': '의석을 정돈하여 주시기 바랍니다. 성원이 되었으...,{'AGENDA_1': {'evidence': {'0': ['의석을 정돈하여 주시기...
1,1001,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,완주,5,{'AGENDA_1': {'0': '의석을 정돈하여 주시기 바랍니다. 성원이 되었으...,{'AGENDA_1': {'evidence': {'0': ['의석을 정돈하여 주시기...
2,1002,제210회 완주군의회(임시회) 제 1 차 본회의회의록,완주,4,{'AGENDA_1': {'0': '의석을 정돈하여 주시기 바랍니다. 성원이 되었으...,{'AGENDA_1': {'evidence': {'0': ['의석을 정돈하여 주시기...
3,1003,제229회 완주군의회(임시회) 제1차 본 회 의 회 의 록,완주,4,{'AGENDA_1': {'0': '의석을 정돈하여 주시기 바랍니다. 성원이 되었으...,{'AGENDA_1': {'evidence': {'0': ['의석을 정돈하여 주시기...
4,1004,제178회 완주군의회(임시회) 제 1 차 본회의회의록,완주,4,{'AGENDA_1': {'0': '좌석을 정돈하여 주시기 바랍니다. 성원이 되었으...,{'AGENDA_1': {'evidence': {'0': ['좌석을 정돈하여 주시기...


## Title_context

In [32]:
# making title + context data
real_id=[]
for i in range(0,len(train)):
  for j in range(1,train.iloc[i,3]+1):
    real_id.append('id'+str(train['id'][i])+'-AGENDA_'+str(j))

real_context=[]
for i in range(0,len(train)):
  for j in range(1,train.iloc[i,3]+1):
    real_context.append(' '.join(list(train['context'][i][f'AGENDA_{j}'].values())))

real_title=[]
for i in range(0,len(train)):
  for j in range(1,train.iloc[i,3]+1):
    real_title.append(train['title'][i])

real_label=[]
for i in range(0,len(train)):
  for j in range(1,train.iloc[i,3]+1):
    real_label.append(train['label'][i][f'AGENDA_{j}']['summary'])

In [33]:
train_real=pd.DataFrame({'id':real_id,'title':real_title,'context':real_context,'summary':real_label})

train_real=process_basic(train_real,'context')
train_real=process_detail(train_real,'context')

train_real['title_context']=train_real['title']+': '+train_real['context']

In [35]:
# Special case preprocessing previously defined
train_real=election(train_real, 'title_context')
train_real=monetary(train_real, 'title_context',tokenizer)

for i in range(len(train_real)):
  train_real.at[i,'title_context']=re.sub(' +',' ',train_real['title_context'][i])

train_real[['id','title_context','summary']].head()

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,id,title_context,summary
0,id1000-AGENDA_1,제207회 완주군의회(임시회) 제1차 본회의: 의석을 정돈하여 주시기 바랍니다. 성...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.
1,id1000-AGENDA_2,제207회 완주군의회(임시회) 제1차 본회의: 먼저 의사일정 제1항 제207회 완주...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.
2,id1000-AGENDA_3,제207회 완주군의회(임시회) 제1차 본회의: 다음은 의사일정 제2항 제207회 완...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.
3,id1000-AGENDA_4,제207회 완주군의회(임시회) 제1차 본회의: 다음은 의사일정 제3항 본회의 휴회의...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...
4,id1001-AGENDA_1,제251회 완주군의회(제1차 정례회) 제1차 본회의: 의석을 정돈하여 주시기 바랍니...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.


In [ ]:
train_real.to_csv(data_path+'train_real_final.csv',index=False)

## Evi_context

In [36]:
# making title + evidence text data
# will be used for additional train data
real_id=[]
for i in range(0,len(train)):
  for j in range(1,train.iloc[i,3]+1):
    real_id.append('id'+str(train['id'][i])+'-AGENDA_'+str(j))

real_title=[]
for i in range(0,len(train)):
  for j in range(1,train.iloc[i,3]+1):
    real_title.append(train['title'][i])

real_label=[]
for i in range(0,len(train)):
  for j in range(1,train.iloc[i,3]+1):
    real_label.append(train['label'][i][f'AGENDA_{j}']['summary'])

real_evi=[]
for i in range(0,len(train)):
  for j in range(1,train.iloc[i,3]+1):
    real_evi.append(train['label'][i][f'AGENDA_{j}']['evidence'].values())

real_num=[]
for i in range(0,len(train)):
  for j in range(1,train.iloc[i,3]+1):
    real_num.append(train['label'][i][f'AGENDA_{j}']['evidence'].keys())

In [40]:
evi_real=pd.DataFrame({'id':real_id,'title':real_title,'evidence':real_evi, 'evi_num':real_num,'summary':real_label})
evi_real['evi_num']=evi_real['evi_num'].apply(len)

In [41]:
# link the evidence text to single paragraph
evidence_text=[]

for i in range(0,len(evi_real)):
  l=[]
  for j in range(0,evi_real['evi_num'][i]):
    l.append(' '.join(list(evi_real['evidence'][i])[j]))
    
  evidence_text.append(' '.join(l))

evi_real['evidence_text']=evidence_text
evi_real=evi_real.drop(['evidence','evi_num'],axis=1)

In [42]:
evi_real=process_basic(evi_real,'evidence_text')
evi_real=process_detail(evi_real,'evidence_text')

evi_real['title_evidencetext']=evi_real['title']+': '+evi_real['evidence_text']
evi_real.columns=['id','title','summary','context','title_context']

evi_real[['id','title_context','summary']].head()

,id,title_context,summary
0,id1000-AGENDA_1,제207회 완주군의회(임시회) 제1차 본회의: 의석을 정돈하여 주시기 바랍니다. 성...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.
1,id1000-AGENDA_2,제207회 완주군의회(임시회) 제1차 본회의: 먼저 의사일정 제1항 제207회 완주...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.
2,id1000-AGENDA_3,제207회 완주군의회(임시회) 제1차 본회의: 다음은 의사일정 제2항 제207회 완...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.
3,id1000-AGENDA_4,제207회 완주군의회(임시회) 제1차 본회의: 다음은 의사일정 제3항 본회의 휴회의...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...
4,id1001-AGENDA_1,제251회 완주군의회(제1차 정례회) 제1차 본회의: 의석을 정돈하여 주시기 바랍니...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.


In [ ]:
# concat all train data
train_real=pd.read_csv(data_path+'train_real_final.csv')
train_evi_concat=pd.concat([train_real,evi_real]).reset_index(drop=True)

train_evi_concat.to_csv(data_path+'train_evi_concat_final.csv',index=False)

# Make test data

In [43]:
test=pd.read_json(data_path+'test.json')
test.head()

,id,title,region,num_agenda,context
0,2000,제235회 본회의 제1차(2012.06.21.),음성,6,"{'AGENDA_1': {'0': '의석을 정돈하여 주시기 바랍니다.', '1': ..."
1,2001,제322회 본회의 제1차(2020.04.22.),음성,11,"{'AGENDA_1': {'0': '의석을 정돈하여 주시기 바랍니다.', '1': ..."
2,2002,제296회 본회의 제1차(2018.02.12.),음성,13,"{'AGENDA_1': {'0': '의석을 정돈하여 주시기 바랍니다.', '1': ..."
3,2003,제72회 전라북도 완주군의회 (임시회) 제1호 본 회 의 회 의 록,완주,5,{'AGENDA_1': {'0': '좌석을 정돈하여 주시기 바랍니다. 성원이 되었으...
4,2004,제76회 본회의 제1차(1998.07.10.),음성,4,{'AGENDA_1': {'0': '의사일정 제1항 의장·부의장 선거를 상정합니다....


In [44]:
real_id=[]
for i in range(0,len(test)):
  for j in range(1,test.iloc[i,3]+1):
    real_id.append('id_'+str(test['id'][i])+'-AGENDA_'+str(j))

real_title=[]
for i in range(0,len(test)):
  for j in range(1,test.iloc[i,3]+1):
    real_title.append(test['title'][i])

real_context=[]
for i in range(0,len(test)):
  for j in range(1,test.iloc[i,3]+1):
    real_context.append(' '.join(list(test['context'][i][f'AGENDA_{j}'].values())))

In [46]:
test_real=pd.DataFrame({'id':real_id,'title':real_title,'context':real_context})

test_real=process_basic(test_real,'context')
test_real=process_detail(test_real,'context')

test_real['title_context']=test_real['title']+': '+test_real['context']

test_real=election(test_real, 'title_context')
test_real=monetary(test_real, 'title_context', tokenizer)

test_real[['id','title_context']].head()

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,id,title_context
0,id_2000-AGENDA_1,제235회 본회의 제1차: 의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금...
1,id_2000-AGENDA_2,"제235회 본회의 제1차: 의사일정 제1항, 음성군의회 제235회 제1차 정례회 회..."
2,id_2000-AGENDA_3,"제235회 본회의 제1차: 의사일정 제2항, 회의록 서명의원 선출의 건을 상정합니다..."
3,id_2000-AGENDA_4,"제235회 본회의 제1차: 의사일정 제3항, 예산결산특별위원회 구성의 건을 상정합니..."
4,id_2000-AGENDA_5,"제235회 본회의 제1차: 의사일정 제4항, 환경분야 현지확인 특별위원회 구성결의안..."


In [ ]:
test_real.to_csv(data_path+'test_evi_final.csv',index=False)